# Student Information

Wangyang He


UIN 625004872


Nickname: heswaggy

Project submission 6

In [1]:
import os
import numpy as np
import cv2
import json
import keras
from keras.models import load_model
from tqdm.notebook import tqdm

In [2]:
!pip install --upgrade youtube_dl

     |████████████████████████████████| 1.9MB 6.1MB/s 


In [3]:
test_videos = '/content/applyvideos'
os.mkdir(test_videos)

In [11]:
import datetime
import youtube_dl, subprocess

#these are the videos link to test, uncomment to download them
urls = [#'https://www.youtube.com/watch?v=lHzTcRqSOhc',
#         'https://www.youtube.com/watch?v=Fh48C5ZgpwQ',
#         'https://www.youtube.com/watch?v=Uwm9OWUNr2E',
#         'https://www.youtube.com/watch?v=ZtYgz3voKs0',
#         'https://www.youtube.com/watch?v=Q938wNKSCs4',
#         'https://www.youtube.com/watch?v=ddpR7tgf5E4',
#         'https://www.youtube.com/watch?v=7izdDkr-RDI',
#         'https://www.youtube.com/watch?v=fSJ3vfDnEtY',
#         'https://www.youtube.com/watch?v=5gRCtYn2HVA',
#         'https://www.youtube.com/watch?v=ZyhdHD6Tkfo',
#         'https://www.youtube.com/watch?v=cI59_PCnVOo',
#         'https://www.youtube.com/watch?v=FZWdJFMq_6Y',
         'https://www.youtube.com/watch?v=-MnMeNqaaNw'
#         'https://www.youtube.com/watch?v=Qleu_GreXZw',
#         'https://www.youtube.com/watch?v=CxcvwdD3X0s',
#         'https://www.youtube.com/watch?v=IAtRsvzzwq8',
#         'https://www.youtube.com/watch?v=nB-ksA1hN7k',
#         'https://www.youtube.com/watch?v=l5Ro6AUg-f0',
#         'https://www.youtube.com/watch?v=ZovQUMqT4n0',
#         'https://www.youtube.com/watch?v=J9p5WmM6uqo',
#         'https://www.youtube.com/watch?v=4ENzF1f9n0k',
#         'https://www.youtube.com/watch?v=xBS9IBXlido'
         ]

for i, id in enumerate(urls):
  #save videos to the folder we created above
  true = os.path.join(test_videos,"video.12.mp4".format(i))
  #download videos
  with youtube_dl.YoutubeDL({'format': 'best'}) as ydl:
    try:
      info = ydl.extract_info(id, download=False)
      video = info['entries'][0] if 'entries' in info else info
    except:
      continue
  #parameters
  url = video['url']
  starttime = datetime.timedelta(seconds=0)
  clip_length = datetime.timedelta(seconds=9999)
  subprocess.call(['ffmpeg', '-i', url, '-ss', str(starttime), '-t', str(clip_length), 
                   '-c:v', 'copy', '-c:a', 'copy', true,]) 
  print("finished downloading")

[youtube] -MnMeNqaaNw: Downloading webpage
finished downloading


In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [75]:
model = load_model('/content/drive/MyDrive/Colab Notebooks/Submission 6/cleaningbathroom_Xcep_sub6.h5') #this .h5 model file is also avaliable on my github.tamu repo
#loading trained model
print("model loaded")

model loaded


In [76]:
def check_model(img, video, model): #this function will check the video with our model, and predict accruacy to produce JSON
    #get the video itself
    capture = cv2.VideoCapture(img)
    # get the video frame per second
    fps = capture.get(5) 
    # get all frames in the video
    video_frame = int(capture.get(7)) 
    # get video length in seconds
    length = int(video_frame/fps) 
    # get the number of predictions our model will make on the test video
    num_pred = int(length/2) 
    # frame per prediction
    fpp = video_frame // num_pred
    # length per frame
    dis = fpp // 15
    #count the frames for each 2 second clip
    count = 0
    pred_count = 1
    #some parameters to use in the for loop
    pred_time = []
    final_frame = []
    clips = []
    tmp = []

    for i in tqdm(range(video_frame)): #iterating over all the frames of the video
            #using openCV
            ret, frame = capture.read()
            count = count + 1

            if count == dis:
                count = count * 0
                frame=cv2.resize(frame,(150,150),interpolation=cv2.INTER_AREA)
                tmp.append(frame)
                final_frame.append(frame)
                
            if i == ((pred_count * fpp) - 1):
                inpt = np.array(tmp)
                inpt = inpt.astype('float32')
                inpt -= np.mean(inpt)
                inpt /= np.max(inpt)
                tmp = []
                data = np.expand_dims(inpt, axis=0)
                pred = model.predict(data, batch_size = 1) # verbose = 1
                rate = pred[0][1] #prediction score stored
                time_init = (pred_count - 1) / 0.5 #start time
                time_end = pred_count / 0.5  # end time
                #produce JSON file
                if rate > 0.8: #if prediction score better than 80%, add in the JSON file
                    time_result = ['Target: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                    json_format = {
                        'videoId': "xBS9IBXlido",
                        'type': "segment",
                        'startTime': np.float(time_init),
                        'endTime': np.float(time_end),
                        'observer': 'CSCE636Spring2021-heswaggy-6',
                        'isHuman': False,
                        'confirmedBySomeone': False,
                        'rejectedBySomeone': False,
                        'observation': {
                            'label': 'Cleaning bathroom',
                            'labelConfidence': np.float(rate)
                        }
                    }
                    clips.append(json_format)
                else:
                    time_result = ['Others: ' +  str(time_init) + 'secs - ' + str(time_end) + 'secs.']
                pred_count += 1
                pred_time.append(time_result)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    capture.release()
    cv2.destroyAllWindows()

    with open(video, 'w', encoding='utf-8') as f:
        json.dump(clips, f, indent=4)
    return final_frame

print("finished")

finished


In [14]:
!mkdir ResultOfDetection

In [77]:
final_frame = check_model('/content/applyvideos/video.21.mp4', '/content/ResultOfDetection/ResultOfDetection.21.json', model)

In [78]:
!cp -a '/content/ResultOfDetection/' '/content/drive/MyDrive/Colab Notebooks/Submission 6'